In [1]:
import tflearn


In [2]:
import numpy as np

In [3]:
train_data = np.load("train_maleria_np.npy")

In [4]:
train_label = np.load('train_label_maleria.npy')

In [5]:
num_classes = 2

In [6]:
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression


In [7]:
train_data.astype(np.float32)
train_label.astype(np.float32)

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [8]:
network = input_data(shape=[None,40,40,3])

In [9]:
print(network)

Tensor("InputData/X:0", shape=(?, 40, 40, 3), dtype=float32)


In [10]:
network = conv_2d(network ,32 , 4 ,padding='same' , activation='relu',strides=1 
           , weights_init='uniform_scaling' , bias_init='zeros', regularizer='L2')

print(network)

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Tensor("Conv2D/Relu:0", shape=(?, 40, 40, 32), dtype=float32)


In [11]:
from tflearn.layers.normalization import local_response_normalization

In [12]:
network = max_pool_2d(network, 2)
print(network)
network = local_response_normalization(network)
print(network)

Tensor("MaxPool2D/MaxPool:0", shape=(?, 20, 20, 32), dtype=float32)
Tensor("LocalResponseNormalization/LocalResponseNormalization:0", shape=(?, 20, 20, 32), dtype=float32)


In [13]:
network = conv_2d(network ,64 , 2 ,padding='same' , activation='relu',strides=1
              , regularizer='L2')

print(network)

Tensor("Conv2D_1/Relu:0", shape=(?, 20, 20, 64), dtype=float32)


In [14]:
network = max_pool_2d(network, 2)
print(network)
network = local_response_normalization(network)
print(network)

Tensor("MaxPool2D_1/MaxPool:0", shape=(?, 10, 10, 64), dtype=float32)
Tensor("LocalResponseNormalization_1/LocalResponseNormalization:0", shape=(?, 10, 10, 64), dtype=float32)


In [15]:
network = fully_connected(network, 128, activation='relu')
network = dropout(network, 0.25)
print(network)

network = fully_connected(network, 256, activation='relu')
network = dropout(network, 0.25)
print(network)

network = fully_connected(network, 512, activation='relu')
network = dropout(network, 0.25)
print(network)



Tensor("Dropout/cond/Merge:0", shape=(?, 128), dtype=float32)
Tensor("Dropout_1/cond/Merge:0", shape=(?, 256), dtype=float32)
Tensor("Dropout_2/cond/Merge:0", shape=(?, 512), dtype=float32)


In [16]:
network = fully_connected(network, 2, activation='softmax')
print(network)
network = regression(network, optimizer='adam', learning_rate=0.001,
                     loss='binary_crossentropy', name='target')
print(network)

Tensor("FullyConnected_3/Softmax:0", shape=(?, 2), dtype=float32)
Tensor("FullyConnected_3/Softmax:0", shape=(?, 2), dtype=float32)


In [17]:
model = tflearn.DNN(network, checkpoint_path='maleria-mixnet',
                    max_checkpoints=3, tensorboard_verbose=2,
                    tensorboard_dir="./logs")


In [18]:
model.fit(train_data, train_label, n_epoch=30, validation_set=0.3, shuffle=True,
          show_metric=True, batch_size=500, snapshot_step=5000,
          snapshot_epoch=False)

Training Step: 905  | total loss: 0.72417 | time: 5.214s
| Adam | epoch: 012 | loss: 0.72417 - acc: 0.5032 -- iter: 23500/38581


KeyboardInterrupt: 